In [1]:
from collections import OrderedDict
from types import SimpleNamespace

cfg_dict = {
    "repo_dir": "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/dinov3",
    "hub_name": "dinov3_vitb16",
    "pretrained_weights_path": "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/dinov3/checkpoints/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth",
    "trained_checkpoint": "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/dinov3/results/da3_weight_1_0/eval/training_12499/teacher_checkpoint.pth",
}
cfg = SimpleNamespace(**cfg_dict)


import torch.hub
import torch

encoder = torch.hub.load(
    cfg.repo_dir, cfg.hub_name, source="local", weights=cfg.pretrained_weights_path
)

state = torch.load(cfg.trained_checkpoint, map_location="cpu")
teacher = state.get("teacher")
if teacher is None:
    raise KeyError("trained_checkpoint missing 'teacher' key")
prefix = "backbone."
new_state = OrderedDict(
    (k[len(prefix) :] if k.startswith(prefix) else k, v) for k, v in teacher.items()
)
missing, unexpected = encoder.load_state_dict(new_state, strict=False)



Downloading: "file:///bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/dinov3/checkpoints/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth" to /home/nfs/surgicaldinov2/.cache/torch/hub/checkpoints/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth


100%|██████████| 327M/327M [00:03<00:00, 95.2MB/s] 


In [3]:
inputs = torch.randn(3, 3, 224, 224)

In [4]:
outputs = encoder.get_intermediate_layers(inputs, n = [5, 7, 9, 11])

In [7]:
outputs[0].shape

torch.Size([3, 196, 768])

In [1]:
import webdataset as wds

In [5]:
from data_scared import ScaredDepthDataModule

In [6]:
from dataclasses import dataclass
from omegaconf import DictConfig, OmegaConf

@dataclass
class MyConfig:
    name: str
    version: float
    is_active: bool = True  # 可以有默认值

In [7]:
# 模拟一个 DictConfig 对象
data = {
    "name": "project_alpha",
    "version": 1.5,
    "is_active": False
}
dict_config = OmegaConf.create(data)

# 确认类型
print(f"原始类型: {type(dict_config)}")
# 原始类型: <class 'omegaconf.dictconfig.DictConfig'>

原始类型: <class 'omegaconf.dictconfig.DictConfig'>


In [10]:
cfg_structured = OmegaConf.merge(OmegaConf.structured(MyConfig), dict_config)

# 4. 转换为 Dataclass 实例
app_config_instance = OmegaConf.to_object(cfg_structured)

In [11]:
app_config_instance

MyConfig(name='project_alpha', version=1.5, is_active=False)

In [1]:
from data_scared import ScaredDepthDataModule
from types import SimpleNamespace

In [2]:
cfg={
    
    "backend": "webdataset",  # "webdataset", "hdf5", or "lance"
    # Patterns are interpreted relative to the project root; they can be
    # overridden on the command line if you place shards elsewhere.
    "train_shards": "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED/shards/shard-train-{000000..000506}.tar", # 22928
    "val_shards": "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED/shards/shard-val-{000000..000118}.tar", # 5907
    "num_workers": 8,
    "batch_size": 32,
    # Shorter side fed to torchvision.transforms.Resize
    "image_size": 256,
    "shuffle_buffer": 1000,
    # Depth range used to build the valid mask
    "min_depth": 0.0001,
    "max_depth": 150.0,
    }  

In [3]:
cfg = SimpleNamespace(**cfg)

In [4]:
loader = ScaredDepthDataModule(cfg)

2025-12-12 16:36:15.127 | INFO     | data_scared:__init__:124 - ScaredDepthDataModule initialized (train_shards=/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED/shards/shard-train-{000000..000506}.tar, val_shards=/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED/shards/shard-val-{000000..000118}.tar, image_size=256)


In [5]:
loader.setup('fit')

2025-12-12 16:36:15.148 | INFO     | data_scared:_build_dataset:131 - Building train WebDataset from pattern: /bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED/shards/shard-train-{000000..000506}.tar
2025-12-12 16:36:15.151 | INFO     | data_scared:_build_dataset:131 - Building val WebDataset from pattern: /bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED/shards/shard-val-{000000..000118}.tar


In [6]:
train_loader = loader.train_dataloader()

In [7]:
val_loader = loader.val_dataloader()

In [14]:
batch = next(iter(val_loader))

In [15]:
batch

{'image': tensor([[[[-0.6965, -0.6794, -0.5938,  ..., -1.0733, -1.0904, -1.1075],
           [-0.7308, -0.6623, -0.5596,  ..., -1.0048, -1.0562, -1.0904],
           [-0.7137, -0.6965, -0.6109,  ..., -1.0219, -1.0562, -1.0562],
           ...,
           [ 0.6563,  0.7419,  0.9646,  ..., -1.2617, -1.2788, -1.3130],
           [ 0.8789,  1.0159,  1.4440,  ..., -1.4329, -1.3987, -1.3302],
           [ 0.9303,  1.2728,  1.8037,  ..., -1.5185, -1.5357, -1.4158]],
 
          [[-1.0203, -0.9503, -0.8803,  ..., -1.3354, -1.3529, -1.3704],
           [-1.0553, -0.9503, -0.8452,  ..., -1.2479, -1.3004, -1.3354],
           [-1.0553, -0.9678, -0.8627,  ..., -1.2479, -1.3004, -1.2829],
           ...,
           [-0.0399,  0.0651,  0.2927,  ..., -1.5105, -1.5105, -1.5280],
           [ 0.2052,  0.3452,  0.7829,  ..., -1.8081, -1.7556, -1.6856],
           [ 0.3627,  0.6954,  1.2206,  ..., -1.9482, -1.9307, -1.8081]],
 
          [[-0.7413, -0.6715, -0.6193,  ..., -1.1596, -1.1770, -1.1944],
    